In [1]:
from __future__ import print_function
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
from math import radians, cos, sin, asin, sqrt

###########################
# Problem Data Definition #
###########################
def create_data_model():
  """Stores the data for the problem"""
  data = {}
  # Locations in block units
  _locations = \
          [(12.55,152.89), # depot 	
           (12.46, 156.21), (12.46, 156.21), # locations to visit
           (12.46, 156.21), (12.54, 152.88),
           (12.54, 152.88), (12.58, 150.71),
           (12.59, 152.95), (12.65, 156.16),
           (12.65, 156.16), (12.67, 152.29),
           (12.41, 150.07), (12.36, 152.66),
           (12.36, 152.66), (12.25, 156.83),
           (12.17, 158.41), (12.05, 156.62)]

  demands = [0, # depot
             1, 1, # row 0
             2, 4,
             2, 4,
             8, 8,
             1, 2,
             1, 2,
             4, 4,
             8, 8]

  capacities = [15, 15, 15, 15,15,15]

  # Multiply coordinates in block units by the dimensions of an average city block, 114m x 80m,
  # to get location coordinates.
  data["locations"] = [(l[0], l[1]) for l in _locations]
  data["num_locations"] = len(data["locations"])
  data["num_vehicles"] = 6
  data["depot"] = 0
  data["demands"] = demands
  data["vehicle_capacities"] = capacities
  return data
#######################
# Problem Constraints #
#######################
def manhattan_distance(position_1, position_2):
 # """Computes the Manhattan distance between two points"""

    lat1=position_1[0]
    lon1=position_1[1]
    
    lat2=position_2[0]
    lon2=position_2[1]
  # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 3959 # Radius of earth in miles
    #print (c*r)
    return c * r  
  
def create_distance_callback(data):
  """Creates callback to return distance between points."""
  _distances = {}

  for from_node in range(data["num_locations"]):
    _distances[from_node] = {}
    for to_node in range(data["num_locations"]):
      if from_node == to_node:
        _distances[from_node][to_node] = 0
      else:
        _distances[from_node][to_node] = (
            manhattan_distance(data["locations"][from_node],
                               data["locations"][to_node]))

  def distance_callback(from_node, to_node):
    """Returns the manhattan distance between the two nodes"""
    return _distances[from_node][to_node]

  return distance_callback

def create_demand_callback(data):
    """Creates callback to get demands at each location."""
    def demand_callback(from_node, to_node):
        return data["demands"][from_node]
    return demand_callback

def add_capacity_constraints(routing, data, demand_callback):
    """Adds capacity constraint"""
    capacity = "Capacity"
    routing.AddDimensionWithVehicleCapacity(
        demand_callback,
        0, # null capacity slack
        data["vehicle_capacities"], # vehicle maximum capacities
        True, # start cumul to zero
        capacity)
###########
# Printer #
###########
def print_solution(data, routing, assignment):
    """Print routes on console."""
    total_dist = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {0}:\n'.format(vehicle_id)
        route_dist = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = routing.IndexToNode(index)
            next_node_index = routing.IndexToNode(assignment.Value(routing.NextVar(index)))
            route_dist += manhattan_distance(
                data["locations"][node_index],
                data["locations"][next_node_index])
            route_load += data["demands"][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            index = assignment.Value(routing.NextVar(index))

        node_index = routing.IndexToNode(index)
        total_dist += route_dist
        plan_output += ' {0} Load({1})\n'.format(node_index, route_load)
        plan_output += 'Distance of the route: {0}m\n'.format(route_dist)
        plan_output += 'Load of the route: {0}\n'.format(route_load)
        print(plan_output)
    print('Total Distance of all routes: {0}m'.format(total_dist))
########
# Main #
########
def main():
  """Entry point of the program"""
  # Instantiate the data problem.
  data = create_data_model()
  # Create Routing Model
  routing = pywrapcp.RoutingModel(
      data["num_locations"],
      data["num_vehicles"],
      data["depot"])
  # Define weight of each edge
  distance_callback = create_distance_callback(data)
  routing.SetArcCostEvaluatorOfAllVehicles(distance_callback)
# Add Capacity constraint
  demand_callback = create_demand_callback(data)
  add_capacity_constraints(routing, data, demand_callback)
  # Setting first solution heuristic (cheapest addition).
  search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
  search_parameters.first_solution_strategy = (
      routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
  # Solve the problem.
  assignment = routing.SolveWithParameters(search_parameters)
  if assignment:
    print_solution(data, routing, assignment)
if __name__ == '__main__':
  main()

Route for vehicle 0:
 0 Load(0) ->  7 Load(8) ->  0 Load(8)
Distance of the route: 9.800649282692119m
Load of the route: 8

Route for vehicle 1:
 0 Load(0) ->  14 Load(4) ->  15 Load(12) ->  0 Load(12)
Distance of the route: 747.0400122909105m
Load of the route: 12

Route for vehicle 2:
 0 Load(0) ->  9 Load(1) ->  8 Load(9) ->  0 Load(9)
Distance of the route: 441.22859135286507m
Load of the route: 9

Route for vehicle 3:
 0 Load(0) ->  16 Load(8) ->  3 Load(10) ->  2 Load(11) ->  1 Load(12) ->  0 Load(12)
Distance of the route: 517.8313734107658m
Load of the route: 12

Route for vehicle 4:
 0 Load(0) ->  5 Load(2) ->  13 Load(6) ->  12 Load(8) ->  11 Load(9) ->  6 Load(13) ->  10 Load(15) ->  0 Load(15)
Distance of the route: 387.9224722258891m
Load of the route: 15

Route for vehicle 5:
 0 Load(0) ->  4 Load(4) ->  0 Load(4)
Distance of the route: 1.9311861083080484m
Load of the route: 4

Total Distance of all routes: 2105.7542846714305m
